In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Model

# seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)


def Harmonic_Model():
    input = Input(shape=(1,))
    x = Dense(100, activation="tanh")(input)
    x = Dense(200, activation="tanh")(x)
    x = Dense(200, activation="tanh")(x)
    x = Dense(400, activation="tanh")(x)
    x = Dense(400, activation="tanh")(x)
    x = Dense(400, activation="tanh")(x)
    x = Dense(400, activation="tanh")(x)
    x = Dense(100, activation="tanh")(x)
    out = Dense(1, activation=None)(x)
    model = Model(input, out)
    return model


model = Harmonic_Model()

# Generate the noisy data
x = np.linspace(0, 2, 200) + 0.001 * np.random.randn(30)
y = np.exp(-2 * x) * (np.cos(6 * math.sqrt(11) * x) + (math.sqrt(11) / 33) * np.sin(6 * math.sqrt(11) * x)) + 0.01 * np.random.randn(30)
x_0 = tf.convert_to_tensor(x[:, np.newaxis], dtype=tf.float32)
y_0 = tf.convert_to_tensor(y[:, np.newaxis], dtype=tf.float32)

# Initialize mu
mu = np.random.rand(1)
mu_tf = tf.Variable(mu, dtype=tf.float32) #Telling tensorflow that mu is a trainable parameter


def loss(x_val, y_val):
    with tf.GradientTape() as tape:
        tape.watch(x_val)
        with tf.GradientTape() as tape2:
            tape2.watch(x_val)
            model_pred = model(x_val)
        grad1 = tape2.gradient(model_pred, x_val)
    grad2 = tape.gradient(grad1, x_val)
    value_diff_loss = tf.reduce_mean(tf.square(((model_pred - y_val))))
    physics_loss = tf.reduce_mean(tf.square((grad2 + mu_tf * grad1 + 400 * model_pred)))
    total_loss = value_diff_loss + 0.01*physics_loss # I want the model to focus on the data loss while considering the physics loss
    return total_loss


# Create optimizers
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
mu_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
epochs = 40000

for epoch in range(epochs):
    with tf.GradientTape(persistent=True) as tap:
        current_loss = loss(x_0, y_0)

    # Compute gradients
    dws_and_dbs = tap.gradient(current_loss, model.trainable_variables)
    mu_grad = tap.gradient(current_loss, [mu_tf])

    # Apply gradients
    optimizer.apply_gradients(zip(dws_and_dbs, model.trainable_variables))
    mu_optimizer.apply_gradients(zip(mu_grad, [mu_tf]))

    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {current_loss.numpy()}, mu = {mu_tf.numpy()}")

        # Plot the results
        t_values = np.linspace(0, 2, 500).reshape(-1, 1).astype(np.float32)
        t_values_tf = tf.convert_to_tensor(t_values)
        predictions = model(t_values_tf).numpy()

        plt.plot(t_values, predictions, label="Predicted", color="red")
        plt.scatter(x, y, s=4, label="True", color="green")
        plt.xlabel("Time")
        plt.ylabel("Displacement")
        plt.title("Harmonic Oscillation Prediction using PINN")
        plt.legend()
        plt.show()

with open("mus.txt", "w") as writer: #Save the values of "mu" as .txt file.
    writer.write(f"{mus}")

model.save("coefficient_finder.h5")